<h2>Linear Regression on King County Housing data using TensorFlow<h2>

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

<h3> Reading and exploring the dataset </h3>

In [2]:
#Reading and Exploring the data
df = pd.read_csv('D:\\Karthik Documents\\HousingData\\house_data.csv')

In [3]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [4]:
#dropping column 'id' as it is an identifier and is not meaningful to be considered as a feature
df = df.drop(columns='id')

#converting column 'date' from string to date format and applying ordinal function ie: converting the date to an interger
#representing the number of days since year 1 day 1. 

df['date'] = df['date'].str[:8]
df['date'] =  pd.to_datetime(df['date'], format='%Y%m%d')
import datetime as dt
df['date'] = df['date'].map(dt.datetime.toordinal)

In [5]:
df.tail()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
21608,735374,360000.0,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,735652,400000.0,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,735407,402101.0,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,735614,400000.0,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287
21612,735521,325000.0,2,0.75,1020,1076,2.0,0,0,3,7,1020,0,2008,0,98144,47.5941,-122.299,1020,1357


In [6]:
df.describe()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,735535.193078,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652
std,113.048011,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631
min,735355.000000,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,735436.000000,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000
50%,735522.000000,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000
75%,735646.000000,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,735745.000000,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


<h3> Preparing data for regression </h3>

In [7]:
#we want to develop a model to predict 'price'. Hence we select price as the dependent variable
y_data = df['price']
x_data = df.drop(columns='price')

In [8]:
#splitting the data into train and test data set. We are also scaling the values of all the independent variables
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state = 50)

scaler = MinMaxScaler()
scaler.fit(X_train)

X_train = pd.DataFrame(data=scaler.transform(X_train), columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(data=scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

<h3>Modeling</h3>

In [9]:
import tensorflow as tf

In [10]:
# preparing a list of tensorflow feature columns 
feat_cols = [tf.feature_column.numeric_column(k) for k in list(X_train.columns.values)]

#preparing the model with 10 layers NN
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=10,num_epochs=1000,shuffle=True)
model = tf.estimator.DNNRegressor(hidden_units=[19,19,19,19,19,19,10,5,2,1], feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_model_dir': 'C:\\Users\\Karthik\\AppData\\Local\\Temp\\tmpxzear2iu', '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None}


In [11]:
#training the model
model.train(input_fn=input_func, steps = 20000);

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Karthik\AppData\Local\Temp\tmpxzear2iu\model.ckpt.
INFO:tensorflow:step = 1, loss = 1.74731e+12
INFO:tensorflow:global_step/sec: 279.968
INFO:tensorflow:step = 101, loss = 2.35277e+11 (0.353 sec)
INFO:tensorflow:global_step/sec: 301.982
INFO:tensorflow:step = 201, loss = 2.05964e+11 (0.347 sec)
INFO:tensorflow:global_step/sec: 282.373
INFO:tensorflow:step = 301, loss = 1.72728e+11 (0.354 sec)
INFO:tensorflow:global_step/sec: 275.865
INFO:tensorflow:step = 401, loss = 1.18132e+11 (0.369 sec)
INFO:tensorflow:global_step/sec: 295.911
INFO:tensorflow:step = 501, loss = 2.57663e+11 (0.331 sec)
INFO:tensorflow:global_step/sec: 287.84
INFO:tensorflow:step = 601, loss = 6.86746e+10 (0.354 sec)
INFO:tensorflow:global_step/sec: 283.094
INFO:tensorflow:step = 701, loss = 3.73606e+11 (0.347 sec)
INFO:tensorflow:global_step/sec: 288.103
INFO:tensorflow:step = 801, loss = 5.57909e+10 (0.347 sec)
INFO:t

INFO:tensorflow:global_step/sec: 193.485
INFO:tensorflow:step = 8101, loss = 2.55999e+11 (0.496 sec)
INFO:tensorflow:global_step/sec: 184.345
INFO:tensorflow:step = 8201, loss = 1.93923e+11 (0.536 sec)
INFO:tensorflow:global_step/sec: 206.892
INFO:tensorflow:step = 8301, loss = 1.21471e+11 (0.491 sec)
INFO:tensorflow:global_step/sec: 259.927
INFO:tensorflow:step = 8401, loss = 1.42335e+11 (0.372 sec)
INFO:tensorflow:global_step/sec: 283.899
INFO:tensorflow:step = 8501, loss = 1.04923e+11 (0.353 sec)
INFO:tensorflow:global_step/sec: 263.98
INFO:tensorflow:step = 8601, loss = 3.20586e+11 (0.385 sec)
INFO:tensorflow:global_step/sec: 195.15
INFO:tensorflow:step = 8701, loss = 2.36252e+11 (0.509 sec)
INFO:tensorflow:global_step/sec: 183.11
INFO:tensorflow:step = 8801, loss = 1.1885e+11 (0.544 sec)
INFO:tensorflow:global_step/sec: 292.019
INFO:tensorflow:step = 8901, loss = 1.142e+11 (0.345 sec)
INFO:tensorflow:global_step/sec: 271.335
INFO:tensorflow:step = 9001, loss = 3.95169e+10 (0.373 s

INFO:tensorflow:global_step/sec: 249.666
INFO:tensorflow:step = 16201, loss = 8.63744e+11 (0.401 sec)
INFO:tensorflow:global_step/sec: 275.5
INFO:tensorflow:step = 16301, loss = 1.16619e+11 (0.369 sec)
INFO:tensorflow:global_step/sec: 270.6
INFO:tensorflow:step = 16401, loss = 2.08336e+11 (0.364 sec)
INFO:tensorflow:global_step/sec: 282.743
INFO:tensorflow:step = 16501, loss = 3.75788e+10 (0.354 sec)
INFO:tensorflow:global_step/sec: 272.944
INFO:tensorflow:step = 16601, loss = 7.31889e+11 (0.356 sec)
INFO:tensorflow:global_step/sec: 320.025
INFO:tensorflow:step = 16701, loss = 1.1595e+11 (0.329 sec)
INFO:tensorflow:global_step/sec: 282.342
INFO:tensorflow:step = 16801, loss = 2.96987e+11 (0.348 sec)
INFO:tensorflow:global_step/sec: 293.371
INFO:tensorflow:step = 16901, loss = 6.81545e+11 (0.347 sec)
INFO:tensorflow:global_step/sec: 265.723
INFO:tensorflow:step = 17001, loss = 2.80068e+10 (0.370 sec)
INFO:tensorflow:global_step/sec: 270.723
INFO:tensorflow:step = 17101, loss = 2.0467e+1

In [12]:
#predicting the output for the testing set
predict_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test, batch_size=10, num_epochs=1, shuffle=False)
pred_gen = model.predict(predict_input_func)
predictions = list(pred_gen)

INFO:tensorflow:Restoring parameters from C:\Users\Karthik\AppData\Local\Temp\tmpxzear2iu\model.ckpt-20000


In [13]:
for i in range(0,5): 
    print(predictions[i])

{'predictions': array([ 482007.9375], dtype=float32)}
{'predictions': array([ 257854.078125], dtype=float32)}
{'predictions': array([ 365213.34375], dtype=float32)}
{'predictions': array([ 500121.6875], dtype=float32)}
{'predictions': array([ 507547.875], dtype=float32)}


In [14]:
final_pred = []
for pred in predictions:
    final_pred.append(pred['predictions'])

In [15]:
#finding out the overall error of the predicted values
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, final_pred)**0.5

165549.69106199543

A error of approx 165,550 in price when the average price is 540,000 indicates that the model is extremely poor. 
We can tweak the number of hiden layers, add more steps and try different different regression techniques to fine tune the model